<a href="https://colab.research.google.com/github/neha-duggirala/Yoga-Instructor/blob/master/Model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
ls "/content/drive/My Drive/datasets/Yoga DataSet/numpy_data/ALL1-8/Trail5/"

'(300, 300)-Test-1115_data.npy'  '(300, 300)-Train-6058_data.npy'


In [ ]:
from tqdm import tqdm
import numpy as np
import cv2
import os,time
from random import shuffle
import matplotlib.pyplot as plt
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.python.keras.callbacks import TensorBoard
print ("Imported!")

Imported!


In [ ]:
def convertRGB(numpy_data):
    data = []
    for gray in numpy_data:
          colored= cv2.cvtColor(gray[0],cv2.COLOR_GRAY2RGB)
          data.append([colored,gray[1]])
    data = np.asarray(data)
    return data


def XYsplit(data):
    X= np.asarray([images[0] for images in data])
    y= np.asarray([images[1] for images in data])
    print(X.shape,y.shape,X[0].shape)
    return X,y

def numpy_attach(src,parts):
  #need to change this name
    name = src+"(90, 90)-Train-2397_data"
    data = np.load(name+"0.npy",allow_pickle=True)
    # data/=255.0
    for i in range(1,parts):
        n=name+str(i)+".npy"
        new_data = np.load(n,allow_pickle=True)
        # try:
        #   new_data/=255.0
        # except:
          # print("None Error")
        print(n,len(data),(new_data).shape)
        data = np.concatenate((data,new_data))
      
    return data

def normalize(array,parts):
  start = 0
  end= start+(len(array)//parts)
  print(end)
  data=np.array(array[start:start+end])
  for i in range(1,parts):
    end= start+(len(array)//parts)
    array[start:start+end]=array[start:start+end]/255.0
  return array

In [ ]:


test_path = "/content/drive/My Drive/datasets/Yoga DataSet/numpy_data/ALL1-8/Trail5/(300, 300)-Test-1115_data.npy"
train_path = "/content/drive/My Drive/datasets/Yoga DataSet/numpy_data/ALL1-8/Trail5/(300, 300)-Train-6058_data.npy"


In [ ]:

training_data=np.load(train_path,allow_pickle=True)
test_data=np.load(test_path,allow_pickle=True)



In [ ]:
training_data=convertRGB(training_data)
test_data=convertRGB(test_data)


In [ ]:
trainX,trainY = XYsplit(training_data)
testX,testY = XYsplit(test_data)



(6058, 300, 300, 3) (6058,) (300, 300, 3)
(1115, 300, 300, 3) (1115,) (300, 300, 3)


In [ ]:
print("Normalizing")
trainX =trainX/255.0
testX = testX /255.0

Normalizing


In [ ]:
ls "/content/drive/My Drive/datasets/Yoga DataSet/models/ALL1-8/Trail1/"

'mobileNet_v2-(300,300,3)F.h5'  'VGG16-(300,300,3)E (1).h5'
'VGG16-(300,300,3)B.h5'         'VGG16-(300,300,3)E.h5'
'VGG16-(300,300,3)D.h5'         'VGG16-(90,90,3)A.h5'
'VGG16-(300,300,3)D.tflite'


In [ ]:
model_path = "/content/drive/My Drive/datasets/Yoga DataSet/models/ALL1-8/Trail1/VGG16-(300,300,3)D.h5"

In [ ]:
from tensorflow.keras.models import load_model
model=load_model(model_path)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0   

In [ ]:
model.evaluate(testX,np.array(testY))


1115/1115 [==============================] - 59s 53ms/sample - loss: 1.6040 - acc: 0.6709


[1.6040122705724742, 0.670852]

In [ ]:
history = model.fit(trainX, trainY, 
                    batch_size=128,
                    epochs=5,
                    verbose=1,
                    # callbacks=[es],
                    validation_split=0.2
                    )

Train on 4846 samples, validate on 1212 samples
Epoch 1/5
4846/4846 [==============================] - 336s 69ms/sample - loss: 1.8748 - acc: 0.5134 - val_loss: 1.7089 - val_acc: 0.4125
Epoch 2/5
4846/4846 [==============================] - 328s 68ms/sample - loss: 1.2402 - acc: 0.5634 - val_loss: 1.2311 - val_acc: 0.5635
Epoch 3/5
4846/4846 [==============================] - 327s 67ms/sample - loss: 1.0334 - acc: 0.6294 - val_loss: 1.0463 - val_acc: 0.6081
Epoch 4/5
4846/4846 [==============================] - 326s 67ms/sample - loss: 0.9020 - acc: 0.6816 - val_loss: 1.1662 - val_acc: 0.6180
Epoch 5/5
4846/4846 [==============================] - 326s 67ms/sample - loss: 0.8206 - acc: 0.7105 - val_loss: 0.9469 - val_acc: 0.6733


In [ ]:
history = model.fit(trainX, trainY, 
                    batch_size=128,
                    epochs=5,
                    verbose=1,
                    # callbacks=[es],
                    validation_split=0.2
                    )

Train on 4846 samples, validate on 1212 samples
Epoch 1/5
4846/4846 [==============================] - 326s 67ms/sample - loss: 0.7634 - acc: 0.7307 - val_loss: 0.8907 - val_acc: 0.6997
Epoch 2/5
4846/4846 [==============================] - 325s 67ms/sample - loss: 0.7216 - acc: 0.7551 - val_loss: 1.1744 - val_acc: 0.5990
Epoch 3/5
4846/4846 [==============================] - 325s 67ms/sample - loss: 0.7325 - acc: 0.7346 - val_loss: 1.1534 - val_acc: 0.5842
Epoch 4/5
4846/4846 [==============================] - 325s 67ms/sample - loss: 0.6861 - acc: 0.7577 - val_loss: 1.1746 - val_acc: 0.5825
Epoch 5/5
4846/4846 [==============================] - 325s 67ms/sample - loss: 0.6656 - acc: 0.7648 - val_loss: 0.9629 - val_acc: 0.6625
